# import libraries

In [1]:
# !pip3 install pandas  
# !pip install scikit-image --no-deps


In [2]:
import numpy as np
from scipy.ndimage import gaussian_filter, label
from skimage.feature import peak_local_max
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from skimage import io
# from skimage import filters, morphology, measure, feature, color
# from skimage.segmentation import watershed
from scipy import ndimage as ndi
import csv
import os
from scipy.spatial.distance import cdist


# functions

In [3]:
def detect_cells(density_map, sigma=2, threshold_factor=1.5):
    """
    Detect cells using local maxima detection on a smoothed density map.

    Parameters:
        density_map (numpy.ndarray): Input density map (2D array).
        sigma (float): Standard deviation for Gaussian smoothing.
        threshold_factor (float): Factor to determine threshold for local maxima based on mean and std.

    Returns:
        cell_centroids (list of tuples): List of detected cell centroids as (x, y) coordinates.
    """
    # Smooth the density map using a Gaussian filter
    smoothed_map = gaussian_filter(density_map, sigma=sigma)

    # Calculate threshold based on the mean and standard deviation of the smoothed map
    mean_val = np.mean(smoothed_map)
    std_val = np.std(smoothed_map)
    threshold = mean_val + threshold_factor * std_val

    # Identify local maxima above the threshold
    coordinates = peak_local_max(smoothed_map, min_distance=3, threshold_abs=threshold)

    # Convert to a list of (x, y) tuples
    cell_centroids = [[int(x), int(y)] for y, x in coordinates]

    return cell_centroids

In [4]:

def calculate_iou_and_matches(predictions, ground_truth, tolerance=5):
    """
    Calculate IoU and return matches between predicted and ground truth centroids.
    
    Parameters:
        predictions (ndarray): Array of predicted centroids (x, y).
        ground_truth (ndarray): Array of ground truth centroids (x, y).
        tolerance (float): Radius within which a match is considered valid.
    
    Returns:
        float: IoU value.
        list: Matched pairs of indices [(pred_idx, gt_idx), ...].
    """
    distances = cdist(predictions, ground_truth, metric='euclidean')
    matched_pairs = []
    matched_pred = set()
    matched_gt = set()
    
    for i, row in enumerate(distances):
        for j, dist in enumerate(row):
            if dist <= tolerance and i not in matched_pred and j not in matched_gt:
                matched_pred.add(i)
                matched_gt.add(j)
                matched_pairs.append((i, j))
    
    tp = len(matched_pairs)
    fp = len(predictions) - tp
    fn = len(ground_truth) - tp
    iou = tp / (tp + fp + fn)
    return iou, matched_pairs

In [5]:
def plot_centroids_with_circles_plt(predictions, ground_truth, matches, tolerance=5):
    """
    Plot predicted and ground truth centroids, highlight matches with green circles and lines.
    
    Parameters:
        predictions (ndarray): Array of predicted centroids (x, y).
        ground_truth (ndarray): Array of ground truth centroids (x, y).
        matches (list): Matched pairs of indices [(pred_idx, gt_idx), ...].
        tolerance (float): Radius of the green circle for matched pairs.
    """
    plt.figure(figsize=(8, 8))
    predictions = np.array(predictions)
    ground_truth = np.array(ground_truth)
    
    # Plot predictions
    plt.scatter(predictions[:, 0], predictions[:, 1], c='blue', label='Predicted Centroids', s=100)
    
    # Plot ground truth
    plt.scatter(ground_truth[:, 0], ground_truth[:, 1], c='red', label='Ground Truth Centroids', s=100)
    
    # Highlight matches
    for pred_idx, gt_idx in matches:
        pred_point = predictions[pred_idx]
        gt_point = ground_truth[gt_idx]
        
        # Connect matched pairs with dashed green lines
        plt.plot([pred_point[0], gt_point[0]], [pred_point[1], gt_point[1]], 
                 c='green', linestyle='--', linewidth=1)
        
        # Draw green circle around the pair
        circle = plt.Circle(((pred_point[0] + gt_point[0]) / 2, 
                             (pred_point[1] + gt_point[1]) / 2), 
                             radius=tolerance, color='green', fill=False, linewidth=1.5)
        plt.gca().add_artist(circle)
    
    # Add labels, legend, and title
    plt.legend()
    plt.title('Centroid Locations with Matches Highlighted')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.grid(False)
    plt.axis('equal')  # Ensure equal aspect ratio for circles
    plt.savefig('C:/Users/narges/PycharmProjects3/pythonProject3/IEEE_transaction_paper/temp.png')

    plt.show()


In [6]:
import cv2
import numpy as np

def plot_centroids_with_circles_cv2(predictions, ground_truth, matches, save_address, tolerance=5):
    """
    Plot predicted and ground truth centroids, highlight matches with green circles and lines using OpenCV.
    
    Parameters:
        predictions (ndarray): Array of predicted centroids (x, y).
        ground_truth (ndarray): Array of ground truth centroids (x, y).
        matches (list): Matched pairs of indices [(pred_idx, gt_idx), ...].
        tolerance (float): Radius of the green circle for matched pairs.
    """
    # Ensure predictions and ground_truth are numpy arrays
    predictions = np.array(predictions)
    ground_truth = np.array(ground_truth)
    
    # Rescale coordinates from 256x256 to 800x800
    scale_factor = 800 / 256
    predictions = predictions * scale_factor
    ground_truth = ground_truth * scale_factor
    
    # Print the range of coordinates for debugging
    print("Predictions range (min, max):", predictions.min(axis=0), predictions.max(axis=0))
    print("Ground truth range (min, max):", ground_truth.min(axis=0), ground_truth.max(axis=0))
    
    # Create a blank white image for plotting
    img = np.ones((800, 800, 3), dtype=np.uint8) * 255
    
    # Draw predictions (blue)
    for pred in predictions:
        # Clamp coordinates to the image size (0 to 799)
        pred = np.clip(pred.astype(int), 0, 799)
        cv2.circle(img, tuple(pred), 5, (255, 0, 0), -1)
    
    # Draw ground truth (red)
    for gt in ground_truth:
        # Clamp coordinates to the image size (0 to 799)
        gt = np.clip(gt.astype(int), 0, 799)
        cv2.circle(img, tuple(gt), 5, (0, 0, 255), -1)
    
    # Highlight matches
    for pred_idx, gt_idx in matches:
        pred_point = predictions[pred_idx]
        gt_point = ground_truth[gt_idx]
        
        # Draw green circle around the pair (midpoint of predicted and ground truth points)
        midpoint = ((pred_point[0] + gt_point[0]) // 2, (pred_point[1] + gt_point[1]) // 2)
        midpoint = np.clip(midpoint, 0, 799)  # Clamp midpoint coordinates
        
        # Ensure midpoint is in integer form
        midpoint = tuple(map(int, midpoint))
        
        # Draw the circle
        cv2.circle(img, midpoint, 10, (0, 255, 0), 1)
    
    # Display the image
    # cv2.imshow("Centroid Locations with Matches Highlighted", img)
    
    # Save the image
    cv2.imwrite(save_address, img)
    return img


# Calculate IoU on real dataset Light-U-Net

In [7]:
data =[]
whole_iou=[]
# Example usage
method= ['light_u_net1' , 'u_net' , 'light_u_net1' , 'u_net']
dataset_type=['synth' , 'synth', 'real', 'real']
for i in range(4):
    input_image_folder=  'C:/Users/narges/PycharmProjects3/pythonProject3/IEEE_transaction_paper/'+dataset_type[i]+'_dataset/outputs_'+method[i]+'/'
    csv_address='C:/Users/narges/PycharmProjects3/pythonProject3/IEEE_transaction_paper/evaluation methods/IoU.csv'
    
    print(csv_address)
    data=[]
    filenames= os.listdir(input_image_folder)
    for file_name in filenames :
        print(file_name)
        # Load or generate a sample density map (replace with your own data)
        # For demonstration, we'll create a synthetic density map
        image_path   = input_image_folder + str(file_name) + '/image_pr_centroids.jpg'
        gt_path   = input_image_folder + str(file_name) + '/image_gt_centroids.jpg'
        real_counts  = input_image_folder + str(file_name) + '/counts.txt'
        file_path    = input_image_folder + str(file_name) + '/iou_'+file_name+ "_"+method[i]+'.txt'
        address= input_image_folder + str(file_name) + '/iou_'+file_name+ "_"+method[i]+'.jpg'
    
        density_map = io.imread(image_path, as_gray=True)
        gt_image = io.imread(gt_path, as_gray=True)
    
        
        # Convert and resize density_map
        density_map_pil = Image.fromarray((density_map * 255).astype('uint8'))  # Convert grayscale to 8-bit image
        density_map_resized = density_map_pil.resize((256, 256))
        density_map_resized = np.array(density_map_resized) / 255.0  # Rescale back to [0, 1]
        
        # Convert and resize gt_image
        gt_image_pil = Image.fromarray((gt_image * 255).astype('uint8'))  # Convert grayscale to 8-bit image
        gt_image_resized = gt_image_pil.resize((256, 256))
        gt_image_resized = np.array(gt_image_resized) / 255.0  # Rescale back to [0, 1]
    
    
        
            # print(density_map_resized.shape , gt_image_resized.shape)
            # Detect cells
        cell_centroids = detect_cells(density_map_resized, sigma=2, threshold_factor=1.5)
        gt_locations = detect_cells(gt_image_resized, sigma=2, threshold_factor=1.5)
        # print(cell_centroids)
        # print(10*"*")
        # print(gt_locations)
        # # Example data
        predictions = cell_centroids  # Predicted centroids
        ground_truth = gt_locations  # Ground truth centroids
        
        # Calculate IoU and matches
        iou, matches = calculate_iou_and_matches(predictions, ground_truth, tolerance=5)
        print(f"IoU: {iou:.4f}")
        print("Matched pairs:", matches)
    
        # Plot the centroids with green circles around matched pairs
        iou_plot= plot_centroids_with_circles_cv2(predictions, ground_truth, matches, address , tolerance=5)
    
        print("*"*10)
        print("image : " , file_name )
        print( f"IoU: {iou}")  # Subtract 1 to ignore the background component
        data.append([file_name , iou])
    
        with open(file_path, 'a') as file:
            file.write("IoU: "+ str(iou) + '\n')
    
    
        # Visualize the results
        plt.figure(figsize=(8, 8))
        plt.subplot(1, 2, 1)
        plt.title("IoU")
        plt.imshow(iou_plot, cmap="hot")
        plt.close()
    if  len(data)<80:
        for k in range(80-len(data)):
            data.append(["0","0"])
    
    if i == 0:
        whole_iou = np.array(data)
    else:
        # Concatenate along axis=1 (add a new column for each method)
        whole_iou = np.concatenate((whole_iou, np.array(data)), axis=1)
        
# df = pd.DataFrame(data, columns=['Image name','IoU',  ])
# df.to_csv(csv_address, index=False)
# Column headers
headers = ['Image name', 'IoU_light_unet_synth' , 'Image name' , 'IoU_unet_synth'  , 'Image name' , 'IoU_light_unet_real'  ,  'Image name'  ,'IoU_unet_real' ]

# CSV file path

# Write data to CSV file
with open(csv_address, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header row
    writer.writerow(headers)
    
    # Write data rows
    writer.writerows(whole_iou)

print(f"Data has been saved to {csv_address}")

  


C:/Users/narges/PycharmProjects3/pythonProject3/IEEE_transaction_paper/evaluation methods/IoU.csv
001_
IoU: 0.8983
Matched pairs: [(0, 26), (1, 3), (2, 2), (3, 35), (4, 39), (5, 111), (6, 101), (7, 64), (8, 56), (9, 55), (10, 37), (11, 31), (12, 16), (13, 97), (14, 42), (15, 94), (16, 98), (17, 110), (18, 90), (19, 50), (20, 79), (21, 45), (22, 109), (23, 93), (24, 0), (25, 82), (26, 1), (27, 112), (28, 107), (29, 72), (30, 81), (31, 114), (32, 40), (33, 8), (34, 70), (35, 44), (36, 18), (37, 66), (38, 10), (39, 48), (40, 54), (41, 106), (42, 65), (43, 43), (44, 15), (45, 14), (46, 52), (47, 58), (48, 92), (49, 87), (50, 96), (51, 38), (52, 49), (53, 62), (54, 74), (55, 17), (56, 113), (57, 57), (58, 95), (59, 88), (60, 34), (61, 85), (62, 19), (63, 30), (64, 76), (65, 75), (66, 68), (67, 91), (68, 60), (69, 22), (70, 36), (71, 89), (72, 105), (73, 7), (74, 102), (75, 84), (76, 108), (77, 73), (78, 77), (79, 59), (80, 117), (81, 115), (82, 4), (83, 53), (84, 29), (85, 51), (86, 80), (8